In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, root_dir, class_path, transform=None):
        super(MyDataset, self
             ).__init__()
        self.root_dir = root_dir
        self.class_path = class_path
        self.transform = transform
        
        # get image list
        all_list =  [os.path.splitext(f) for f in os.listdir(self.root_dir)] # jpg + txt
#         assert [f for f,e in all_list if e=='.jpg'] == [f for f,e in all_list if e=='.txt'], "num_image != num_txt"
        self.data_list = [f for f,e in all_list if e=='.jpg']
        
        # get class list
        with open(class_path, 'r', encoding='utf-8') as f:
            self.classes = [c for c in f.read().split('\n')]
        self.class_to_idx = {self.classes[i]: i for i in range(len(self.classes))}

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        image_path = os.path.join(self.root_dir, self.data_list[idx]+'.jpg')
        ann_path = os.path.join(self.root_dir, self.data_list[idx]+'.txt')
        img = Image.open(image_path)
        with open(ann_path, 'r') as f:
            anno = [a.split(' ') for a in f.read().strip().split('\n')]
        anno = np.array(anno)
        labels = anno[:,0].astype(np.int) #- 10
        bboxes_yolo = anno[:,1:].astype(np.float) # centerX, centerY, w, h [ratio] - yolo style
        bboxes = bboxes_yolo.copy() # x,y,w,h

        if self.transform:
            img = self.transform(img).unsqueeze(0)
            
        bboxes[:,0] = (bboxes_yolo[:,0] - bboxes_yolo[:,2]/2)*img.shape[3] # x
        bboxes[:,1] = (bboxes_yolo[:,1] - bboxes_yolo[:,3]/2)*img.shape[2] # y
        bboxes[:,2] = bboxes_yolo[:,2]*img.shape[3]
        bboxes[:,3] = bboxes_yolo[:,3]*img.shape[2]
        bboxes = bboxes.astype(np.int)

        return img, labels, bboxes, bboxes_yolo

---
# Classification

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os, shutil
from time import sleep
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from torch.distributions.multivariate_normal import MultivariateNormal
from tqdm import tqdm_notebook as tqdm
# import mymodels
# from mymodels.BN_Inception import BN_Inception

def my_cos(x,y):
    _x = F.normalize(x, p=2, dim=1)
    _y = F.normalize(y, p=2, dim=1)
    return _x.matmul(_y.transpose(0,1))

class Flatten(torch.nn.Module):
    """
    torch.nn.Sequential에서 사용가능한 flatten 모듈
    """
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)
    
class OracleModel(nn.Module):
    def __init__(self):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(OracleModel, self).__init__()
#         ****torch pretrained net****
#         net = models.shufflenetv2.shufflenet_v2_x1_0(pretrained=True) # 1024
#         net = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)
#         net = models.mobilenet_v2(pretrained=True)
#         net = models.densenet201(pretrained=True)
        net = models.resnet50(pretrained=True)

        modules = list(net.children())[:-2]      # resnet conv_5
        self.backbone = torch.nn.Sequential(*modules) # 2048
        avg_pool = torch.nn.AdaptiveAvgPool2d(output_size=(1, 1))
        flatten = Flatten() 
        
        # fc layer
        self.fc = nn.Sequential(
            avg_pool,
            flatten
        ) # 2048
    
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.topk = 3
        self.threshold = 0.75
        self.feature_len = 2048

        self.sort_order_descending = False
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        self.roi_upsample = nn.UpsamplingBilinear2d([224,224])
        self.roi_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ])
        
    def embed(self, images):
        r = self(images)
        return self.fc(r)

    def forward(self, images):
        """Extract feature vectors from input images."""
        return self.backbone(images)
#         return self.backbone(images)[3]    
    

    def makeAllReference_online(self, referenceImgDirPath):
        """
        임베딩 디비 생성... flask 서버용 
        utils.makeAllReferenceCSV 에서 csv 저장만 안함
        """
        reference_dataset = torchvision.datasets.ImageFolder(
            root=referenceImgDirPath,
            transform=transforms.Compose([
#                 transforms.Resize([224,224]),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
        reference_loader = torch.utils.data.DataLoader(
            reference_dataset,
            batch_size=32,
            num_workers=0,
            shuffle=False
        )
        
        # get all data and input to model
        _temp = []
        for data, target in tqdm(reference_loader):
            outputs = self.embed(data.to(self.device)).data # .data 안하면 메모리 오버플로남...
            _temp.append(outputs)

        represented = torch.cat(_temp, dim=0)
        # raw data label 별 평균 구해두기
        self.df_ref_online = pd.DataFrame(represented.cpu().numpy())
        self.df_ref_online['label'] = [reference_dataset.classes[i] for i in reference_dataset.targets]
        self.reference_means_online = self.df_ref_online.groupby('label').mean()
        self.dir_list_online = reference_dataset.classes

        # 즉각 임베딩 디비 생성
        self.setReferenceDataset(self.dir_list_online, self.df_ref_online, self.reference_means_online)
        return self.dir_list_online, self.df_ref_online, self.reference_means_online

    def addNewData_online(self, im_arr, label):
        """
        @params im_arr: h X w X 3 (RGB)
        @params label: "praL"
        """
        data = self.roi_transform(im_arr).unsqueeze(0)
        outputs = self.embed(data.to(self.device)).data
        new_feature = list(outputs.squeeze().cpu().numpy())
        new_feature.append(label)
        new_feature[-1]
        # concat to prev df_ref_online
        self.df_ref_online.loc[self.df_ref_online.shape[0]] = new_feature
        # revise reference_means_online 
        self.reference_means_online = self.df_ref_online.groupby('label').mean()
        # 즉각 임베딩 디비 생성
        self.setReferenceDataset(self.dir_list_online, self.df_ref_online, self.reference_means_online)
        print('[Detector]: addNewData_online -', label)
        return

    def addNewLabel_online(self, dirPath, label):   
        """
        @params dirPath: "server/vu-visor/static/images_ext"
        @params label: "praL"
        """
        finalDirPath = os.path.join(dirPath, label)
        ims = [Image.open(os.path.join(finalDirPath,i)) for i in os.listdir(finalDirPath)] # n_img X h X w X c
        ims_tensor = torch.stack([self.roi_transform(im) for im in ims])
        outputs = self.embed(ims_tensor.to(self.device)).data
        new_features =outputs.cpu().numpy() # n_img X 2048

        new_df_ref_online = pd.DataFrame(new_features)
        new_df_ref_online['label'] = label
        # add new label
        if label not in self.dir_list_online: # 중복된 이름 없을 시 새로 등록
            self.dir_list_online.append(label)
            self.dir_list_online.sort() # 반드시 소팅해서 클래스 번호 재정렬 해줘야함
        # concat to prev df_ref_online
        _df_ref_online_non_overlap = self.df_ref_online[self.df_ref_online['label']!=label] # 중복된 이전 데이터 제거
        self.df_ref_online = pd.concat([_df_ref_online_non_overlap, new_df_ref_online], ignore_index=True)
        # revise reference_means_online 
        self.reference_means_online = self.df_ref_online.groupby('label').mean()
        # 즉각 임베딩 디비 생성
        self.setReferenceDataset(self.dir_list_online, self.df_ref_online, self.reference_means_online)
        print('[Detector]: addNewLabel_online -', label)
        return 
        
    # 임베딩 디비 생성
    def setReferenceDataset(self, sample_dir_list, df_ref_feature_sampled, reference_means_sampled):
        self.reference_classes = sample_dir_list
        self.reference_targets = list(df_ref_feature_sampled.iloc[:,-1])
        self.embedded_features_cpu = torch.tensor(df_ref_feature_sampled.iloc[:,:-1].values).float().data # float64->float32(torch default), cpu
        self.embedded_features = self.embedded_features_cpu.to(self.device) # gpu
        self.embedded_means_numpy = reference_means_sampled.values
        self.embedded_means = torch.tensor(self.embedded_means_numpy).float().data.to(self.device) # float64->float32(torch default), gpu

        self.c2i = {c:i for i,c in enumerate(self.reference_classes)}
        self.embedded_labels = torch.tensor([self.c2i[c] for c in self.reference_targets]).to(self.device)
        
        # make datafames for plot
        # ***필요없음 어차피 pca 해야함
        self.df_ = pd.DataFrame(np.array(self.embedded_features_cpu))
        self.df_['name'] = self.reference_targets
        self.centers_ = pd.DataFrame(self.embedded_means_numpy)
        self.centers_['name'] = self.reference_classes

    def fit_pca(self, n_components=4):
        # pca model fit
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components) # 2048 차원 다쓰면 나중에 샘플링에서 계산 오류남, sample 개수보다 많으면 촐레스키 분해 에러나는듯
        self.transformed = self.pca.fit_transform(self.embedded_features_cpu)

        # show PCA features 
        self.df = pd.DataFrame(self.transformed)
        self.df['name'] = self.reference_targets
        self.centers = pd.DataFrame(self.pca.transform(self.embedded_means_numpy))
        self.centers['name'] = self.reference_classes

    def inference_tensor3(self, inputs, metric='cos', knn=True):
        """
        roi 피처맵을(nxcx7x7) 인풋으로 받음
        @params metric: [l2, mahalanobis, prob]
        @params inputs: # N x C x H x W
        """            
        # inputs shape: Batch*C*H*W
        # input to backbone model
        self.inputs = inputs
        self.outputs = self.fc(self.inputs).data # n_roi X features 2048    

        # inference
        self.sort_order_descending = True         
        if knn: self.dists = my_cos(self.outputs, self.embedded_features)
        else: self.dists = my_cos(self.outputs, self.embedded_means)

        self.dists_sorted = self.dists.sort(dim=1, descending=self.sort_order_descending)
        if knn: self.predicts = self.embedded_labels[self.dists_sorted.indices][:,:self.topk]
        else: self.predicts = self.dists_sorted.indices[:,:self.topk]

        self.predicts_dist = self.dists_sorted.values[:,:self.topk]
        return self.predicts.data, self.predicts_dist.data    
    
    def inference_tensor_texture(self, inputs, metric='cos', knn=True, SUM=True):
        """
        현재 미니배치 1인 경우만 작동
        roi 피처맵을(nxcx7x7) 인풋으로 받음
        @params metric: [l2, mahalanobis, prob]
        @params inputs: # N x C x H x W
        """            
        # inputs shape: Batch*C*H*W
        # input to backbone model
        # 1 X 2048 X 7 X 7 --> 7*7 X 2048 --> 1 X 3
        # n X 2048 X 7 X 7 --> n*7*7 X 2048 --> n X 3
        self.outputs = inputs.data.transpose(1,2).transpose(2,3).reshape(-1,inputs.shape[1])
        # self.outputs = inputs.squeeze().transpose(0,1).transpose(1,2).reshape(-1,inputs.shape[1]).data 
        # 7*7 X features 2048   
        
        # inference
        self.sort_order_descending = True         
        if knn: self.dists = my_cos(self.outputs, self.embedded_features)
        else: self.dists = my_cos(self.outputs, self.embedded_means)

        self.dists_sorted = self.dists.sort(dim=1, descending=self.sort_order_descending)
        if knn: self.predicts = self.embedded_labels[self.dists_sorted.indices][:,:self.topk]
        else: self.predicts = self.dists_sorted.indices[:,:self.topk]

        self.predicts_dist = self.dists_sorted.values[:,:self.topk]

        if(SUM):
    #             ---- dist sum mode ----
            aa = pd.DataFrame(list(zip(self.predicts[:,0].cpu().numpy(), self.predicts_dist[:,0].cpu().numpy())), columns=['label','dist'])
            self.aa = aa
            # pp = aa.groupby('label').sum()
            # pp = pp.sort_values(by='dist', ascending=False)
            # pp = list(pp.index[:3])
            # if(len(pp)!=self.topk):
            #     pp = pp*self.topk
            #     pp = pp[:self.topk]
            
            # 배치별 번호 매기기
            self.aa['batch'] =  np.repeat(np.arange(inputs.shape[0]), inputs.shape[2]*inputs.shape[3])
            # 다중 그루핑
            zz = self.aa.groupby(['batch', 'label']).agg(['sum', 'mean', 'max'])
            # 2중 컬럼 인덱스 해제
            zz.columns = zz.columns.droplevel(0) 
            # sum 순으로 정렬
            zz = zz.reset_index().sort_values(by=['batch', 'sum'], ascending=False).set_index(['batch', 'label'])
            # 배치별 가장 큰 값만 리턴, dist는 선택 레이블의 평균값
            preds = zz.groupby('batch').head(1).sort_index(0).index.get_level_values(level=1).tolist()
            preds = np.repeat(preds, 3).reshape(-1,3)
            return torch.tensor(preds).data
        else:
    #             ---- voting mode ----
            c = Counter(torch.cat([*self.predicts]).cpu().numpy())
            self.c = c
            pp = list(zip(*(c.most_common()[:3])))[0]
            if(len(pp)!=self.topk):
                pp = pp*self.topk
                pp = pp[:self.topk]    
            return torch.tensor([pp]).data        

    def inference_tensor4(self, inputs):
        """
        roi 피처맵을(nxcx7x7) 인풋으로 받음
        """            
        # inputs shape: Batch*C*H*W
        # input to backbone model
        self.inputs = inputs
        self.outputs = self.fc(self.inputs).data # n_roi X features 2048    

        # inference
        self.dists = my_cos(self.outputs, self.embedded_means)
        self.predicts_dist = self.dists
        return self.predicts_dist.data    

    # 각 레이블 별 평균과의 L2 거리 계산 (PCA 적용안함)
    def calc_l2(self, label):
        mu = self.embedded_means[label]
        xx = self.outputs

        # 20*1280 X 1280*20 --diag--> 20
        # num_roi*dim X dim*num_roi --diag--> num_roi
        xx_sub_mu = xx.sub(mu)
        l2_dist = xx_sub_mu.matmul(xx_sub_mu.t()).sqrt().diag()
        return l2_dist.cpu()

    # 각 레이블 별 평균과의 cosine simility 계산 (PCA 적용안함)
    def calc_cos(self, label):
        mu = self.embedded_means[label]
        xx = self.outputs

        # 20*1280 X 1280*20 --diag--> 20
        # num_roi*dim X dim*num_roi --diag--> num_roi
        mu = mu.unsqueeze(0)
        cos_dist = self.cos(xx, mu)
        return cos_dist.cpu()        

    def inference_file(self, imgPath):              
        # imgPath = "./server/oracle_proj/predict.jpg"
        frame = cv2.imread(os.path.join(imgPath), cv2.IMREAD_COLOR)
        return self.inference_tensor(frame)

    def show_img(self, imgPath):
        frame = cv2.imread(os.path.join(imgPath), cv2.IMREAD_COLOR)
        # opencv frme input // H*W*C(BGR)
        # 0-255 3 channel
        inputs = torch.Tensor(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))/255 # 여기 Tensor 소문자로 바꾸면 안됨... 차이 알아보기
        plt.imshow(inputs)
        plt.show()
        
    def show_tensor(self, t):
        plt.imshow(t.numpy().transpose([1,2,0]))
        plt.show()
        
    def plot(self):
        self.fit_pca(n_components=4)
        _outputs = self.pca.transform(self.outputs.cpu())

        plt.figure(figsize=(18,9))
        plt.title("Vector space, pca_n: "+str(self.feature_len))
        ax = sns.scatterplot(x=0, y=1, hue='name', data=self.df, palette="Set1", legend="full")
        ax2 = sns.scatterplot(x=0, y=1, hue='name', data=self.centers, palette="Set1", s=150, legend=None, edgecolor='black')
        plt.scatter(_outputs[:,0], _outputs[:,1], marker='x', c='black')
        plt.show()

    # save plot image for web
    def save_plot(self, plotPath):
        self.fit_pca(n_components=4)
        _outputs = self.pca.transform(self.outputs.cpu())

        plt.figure(figsize=(9,9))
        plt.title("Vector space, pca_n: "+str(self.feature_len))
        ax = sns.scatterplot(x=0, y=1, hue='name', data=self.df, palette="Set1", legend="full", s=30)
        ax2 = sns.scatterplot(x=0, y=1, hue='name', data=self.centers, palette="Set1", s=150, legend=None, edgecolor='black')
        plt.scatter(_outputs[:,0], _outputs[:,1], marker='x', c='black', s=120)
        plt.savefig(plotPath)

In [5]:
model = OracleModel()
model.to(model.device)
model.eval()

OracleModel(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

In [6]:
"""
차후 util로 리팩토링
"""
import PIL
from collections import Counter 


def myRotation(image):
    return TF.rotate(image, 90, expand=True)

def myTencCrop(image):
    return TF.ten_crop(image, 224, vertical_flip=False)

# -------------- 전체 임베딩 피처 디비 생성 --------------
def makeAllReferenceCSV2(referenceImgDirPath, featuresPath, meanPath, n_sample=None):
    """
    전체 임베딩 피처 dataframe 생성 후 csv로 저장
    """
    reference_dataset = torchvision.datasets.ImageFolder(
        root=referenceImgDirPath,
        transform = transforms.Compose([
            transforms.Resize([448,448]),
            transforms.TenCrop([224,224]), # this is a list of PIL Images        
            transforms.Lambda(lambda crops: torch.stack([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(transforms.ToTensor()(crop)) for crop in crops], dim=0)), # returns a 4D tensor    
        ])         
    )


    reference_loader = torch.utils.data.DataLoader(
        reference_dataset,
        batch_size=4,
        num_workers=0,
        shuffle=False
    )

    # get all data and input to model
    _temp = []
    _targets = []
    for data, target in tqdm(reference_loader):
        data = data.reshape(-1, 3,224,224)
        target = target.reshape(-1, 1).repeat(1,10).flatten()
        outputs = model.embed(data.to(model.device)).data # .data 안하면 메모리 오버플로남...
        _temp.append(outputs)       
        _targets.append(target)

    represented = torch.cat(_temp, dim=0)
    _targets = torch.cat(_targets, dim=0).cpu().numpy().astype(int)
    # raw data label 별 평균 구해두기
    df_ref = pd.DataFrame(represented.cpu().numpy())
    df_ref['label'] = [reference_dataset.classes[i] for i in _targets]

    # sampling
    if(n_sample is not None):
        df_ref_sampled = df_ref.groupby('label').apply(pd.DataFrame.sample, n=n_sample).reset_index(drop=True)
    else: df_ref_sampled = df_ref # no sampling
    reference_means = df_ref_sampled.groupby('label').mean()

    # 자료 저장
    df_ref_sampled.to_csv(featuresPath, encoding='utf8', index=False)
    reference_means.to_csv(meanPath, encoding='utf8', index=True)
    return


# -------------- 전체 임베딩 피처 디비 생성 --------------
def makeAllReferenceCSV(referenceImgDirPath, featuresPath, meanPath, n_sample=None):
    """
    전체 임베딩 피처 dataframe 생성 후 csv로 저장
    """
    reference_dataset = torchvision.datasets.ImageFolder(
        root=referenceImgDirPath,
        transform=transforms.Compose([
#             transforms.RandomRotation(90, expand=True),
            transforms.Resize([224,224]),
#             myRotation,
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomRotation(45),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    )
    reference_loader = torch.utils.data.DataLoader(
        reference_dataset,
        batch_size=32,
        num_workers=0,
        shuffle=False
    )

    # get all data and input to model
    _temp = []
    for data, target in tqdm(reference_loader):
        outputs = model.embed(data.to(model.device)).data # .data 안하면 메모리 오버플로남...
        _temp.append(outputs)       

    represented = torch.cat(_temp, dim=0)
    # raw data label 별 평균 구해두기
    df_ref = pd.DataFrame(represented.cpu().numpy())
    df_ref['label'] = [reference_dataset.classes[i] for i in reference_dataset.targets*1]
    
    # sampling
    if(n_sample is not None):
        df_ref_sampled = df_ref.groupby('label').apply(pd.DataFrame.sample, n=n_sample).reset_index(drop=True)
    else: df_ref_sampled = df_ref # no sampling
    reference_means = df_ref_sampled.groupby('label').mean()

    # 자료 저장
    df_ref_sampled.to_csv(featuresPath, encoding='utf8', index=False)
    reference_means.to_csv(meanPath, encoding='utf8', index=True)
    return


def set_sample_val_img(imgValPath, imgValTempPath, n_class_for_val, musthave_list=[]):
    """
    n개 클래스만 validation 샘플링 하는 함수
    @param imgValPath = '.\\img_real\\val'
    @param imgValTempPath = '.\\img_real\\_val_temp'
    @param n_class_for_val = 20
    @musthave_list = 반드시 포함할 리스트
    """
    all_dir_list = os.listdir(imgValPath)
    sample_dir_list = np.random.permutation(all_dir_list)[:n_class_for_val]
    for i in musthave_list:
        if i not in sample_dir_list: sample_dir_list = np.concatenate([sample_dir_list, [i]])

    # 추출될 이미지 저장 폴더 비우기... 비동기 버그 때문에 sleep넣음 
    if os.path.isdir(imgValTempPath): shutil.rmtree(imgValTempPath)
    sleep(0.1)
    os.makedirs(imgValTempPath)

    # copy sampled dir
    for class_name in tqdm(sample_dir_list):
        if not os.path.isdir(os.path.join(imgValTempPath, class_name)):
            os.mkdir(os.path.join(imgValTempPath, class_name))
        command = ['cmd', '\/c', 'copy',  os.path.join(imgValPath, class_name) , os.path.join(imgValTempPath, class_name)]    
        subprocess.check_output(command)
    sample_dir_list.sort() # 토치 데이터로더도 자동으로 이름순 소팅하므로 꼭 해야함
    return list(sample_dir_list)

def get_sample_reference(featuresPath, meanPath, showData=False):
    """
    n개 클래스 임베딩 & 레이블 & means 추출 
    """
    # load saved embedding features & means
    df_ref = pd.read_csv(featuresPath, encoding='utf8')
    reference_means = pd.read_csv(meanPath, encoding='utf8', index_col='label')
    
    if showData:
        display(df_ref)
        display(reference_means)

    df_ref_featere_sampled = None
    for c in sample_dir_list:
        _df = df_ref[df_ref['label'] == c]
        if df_ref_featere_sampled is None: df_ref_featere_sampled = _df
        else: df_ref_featere_sampled = df_ref_featere_sampled.append(_df)

    df_ref_featere_sampled = df_ref_featere_sampled.reset_index(drop=True)
    reference_means_sampled = reference_means.loc[sample_dir_list]
    return df_ref_featere_sampled, reference_means_sampled

def calcWindowSlidePreds(target_all, preds_all, n_window=7, mode="first"):
    """
    target_all = 정답 리스트
    preds_all = 예측값 리스트 
    mode = 'all' # first, all
    n_window = 3 
    @return new_preds_all = []
    """
    
    # preds, gt 모아서 데이터프레임화
    df = pd.DataFrame(preds_all.numpy())
    df['gt']= target_all.numpy()

    # 클래스별 인덱스 시작점 계산. 데이터로더 셔플하면 안됨
    df_agg = df.groupby('gt').count()
    df_agg['end_idx'] = df_agg[0].cumsum()

    # 윈도 내 데이터 voting
    startidx = 0
    new_preds_all = []
    for i in df_agg.index:
        for idx in range(startidx, df_agg['end_idx'][i]):
            bookmarkidx = idx-n_window+1 if idx-n_window>=0 else 0
            if idx-n_window<startidx:
                bookmarkidx = startidx

            if mode=='all': # 1~3위 까지 전체 사용 모드, [::-1] 은 가장 최근 값을 맨 앞으로 보내려고
                res = np.concatenate(preds_all[bookmarkidx:idx+1].numpy()[::-1])
            elif mode=='first': # 1위만 사용모드       
                res = preds_all[bookmarkidx:idx+1, 0].numpy()[::-1]
            res_cnt = Counter(res) # counter는 먼저나온 값이 먼저 등록됨, 즉 카운터가 동률이면 최신값이 가장 먼저 나옴
#             print(idx-n_window, idx, bookmarkidx, idx+1, res, res_cnt.most_common()[:2], '[Pred]:', res_cnt.most_common()[0][0])
            new_preds_all.append(res_cnt.most_common()[0][0])
#         print('-------------------------')
        startidx = df_agg['end_idx'][i]
    return new_preds_all

In [7]:
# """
# 전체 피처 데이터 재생성
# """
# referenceImgDirPath = './references/images_trimed_10-10shot' # 10 classes

# featuresPath = 'cosmetic_features_cos.csv'
# meanPath = 'cosmetic_means_cos.csv'
# makeAllReferenceCSV(referenceImgDirPath, featuresPath, meanPath)


# # -------------- n개 클래스만 validation 샘플링 세팅 및 폴더정리 ----------------
# reference_dataset = torchvision.datasets.ImageFolder(
#     root=referenceImgDirPath,
#     transform=transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])
# )
# sample_dir_list = reference_dataset.classes

# # -------------- 레퍼런스 데이터에서 n개 클래스 임베딩 & 레이블 & means 추출 -----------------
# featuresPath = 'cosmetic_features_cos.csv'
# meanPath = 'cosmetic_means_cos.csv'

# df_ref_featere_sampled, reference_means_sampled = get_sample_reference(featuresPath, meanPath, showData=False)

# # ---------------------------- 레퍼런스셋 로드 및 임베딩 세팅 ---------------------------------
# model.setReferenceDataset(sample_dir_list, df_ref_featere_sampled, reference_means_sampled)

---
#  
#  
# Detecter Performance

In [8]:
# ------------- non max suppression -------------
# https://wns349.github.io/2018/10/16/nms/
from itertools import combinations, permutations

# Calc IOU
def calcIOU(box1, box2):
    """
    box - x,y,w,h
    """
    area_box1 = box1[2]*box1[3]
    area_box2 = box2[2]*box2[3]
    x1_max = max(box1[0], box2[0])
    x2_min = min(box1[0]+box1[2], box2[0]+box2[2])
    y1_max = max(box1[1], box2[1])
    y2_min = min(box1[1]+box1[3], box2[1]+box2[3])
    
    area_intersection = max(0, x2_min-x1_max) * max(0, y2_min-y1_max)
    area_union = area_box1+area_box2-area_intersection +1e-9
    return area_intersection/area_union

def non_max_sup_one_class(bboxes, threshold=0.2, descending=False):
    """
    @params threshold - 
    @params ascending - 기본이 내림차순,
    """
    bboxes = list(bboxes)
    bboxes.sort(key = lambda x: x[2], reverse=descending) # 거리값이므로 오름차순, 확률이면 내림차순  
    bboxes = np.array(bboxes)
    keeps = [True]*len(bboxes)

    for i, bbox in enumerate(bboxes):
        if not keeps[i]: continue
        for j in range(i+1, len(bboxes)):
            if not keeps[i]: continue
            iou_res = calcIOU(bbox[0], bboxes[j][0])
            if iou_res>threshold: keeps[j] = False
    return bboxes[keeps]

def rpn3(box, n_slice_x, n_slice_y):
    """
    n분할 rpn 조합
    @params box: x,y,w,h
    @params im: nomalized image tensor N x C x W x H
    @return 
    """
    w = box[2]/n_slice_x
    h = box[3]/n_slice_y
    
    rois = []
    boxes = []
    
    coords = np.array(np.meshgrid(list(range(n_slice_x+1)), list(range(n_slice_y+1)))).T.reshape(-1,2)
    
    for a,b in combinations(coords,2):
        if a[0]>=b[0] or a[1]>= b[1]: continue # 넓이 없는 사각형 제거
#         print(a,b)
        boxes.append([box[0]+a[0]*w, box[1]+a[1]*h, 
                      (b[0]-a[0])*w, (b[1]-a[1])*h]) # x,y,w,h

    boxes = torch.tensor(boxes)
    return boxes

def get_rois(images, featuremaps, bboxes, texture_mode=False, n_slice=3):
    """
    roi-align from feature map
    @params images - 원본 이미지 (사이즈 계산용)
    @params featuremaps - CNN Backbone 거쳐 나온 것
    @params bboxes - 원본 이미지에서의 bboxes (x,y,w,h)
    @return 피처맵에서 스케일된 bbox부분 7x7로 roi-align된 피처맵
    """
    # calc bbox ratio
    ratio_y = featuremaps.shape[2]/images.shape[2]
    ratio_x = featuremaps.shape[3]/images.shape[3]
    bboxes_scaled = bboxes.clone()#.detach()

    bboxes_scaled[:,0] = bboxes_scaled[:,0]*ratio_x # for x
    bboxes_scaled[:,1] = bboxes_scaled[:,1]*ratio_y # for y
    bboxes_scaled[:,2] = bboxes_scaled[:,2]*ratio_x # for w
    bboxes_scaled[:,3] = bboxes_scaled[:,3]*ratio_y # for h

    # x,y,w,h -> x1, y1, x2, y2 그래디언트 학습되는 변수가 아니므로 inplace 계산 들어가도 괜찮다
    bboxes_scaled[:, 2] = bboxes_scaled[:, 0] + bboxes_scaled[:, 2]
    bboxes_scaled[:, 3] = bboxes_scaled[:, 1] + bboxes_scaled[:, 3]
    
    crops = torchvision.ops.roi_align(featuremaps, [bboxes_scaled], [7,7])
    return crops

def box_cvt(box):
    """
    auto grad possable
    @params box - N x 4
    x,y,w,h --> x1,y1,x2,y2
    """
    box2 = box.clone().float()
    box2[:,0] = box[:,0]
    box2[:,1] = box[:,1]
    box2[:,2] = box[:,0] + box[:,2]
    box2[:,3] = box[:,1] + box[:,3]
    return box2

In [9]:
from sklearn.mixture import GaussianMixture
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import scipy.stats as stats
import math

def getOrdi(mask, boundary):
    """
    max/min x,y 좌표 구하기
    """
    mask_list = mask.tolist()
    mask_T_list = mask.T.tolist()
    # x1, y1, x2, y2
    x1 = float('inf')
    x2 = float('-inf')
    y1 = float('inf')
    y2 = float('-inf')

    for row in mask_list:
        try:
            xmin = row.index(boundary)
            xmax = len(row)-row[::-1].index(boundary)-1
            if x1>xmin: x1=xmin
            if x2<xmax: x2=xmax
        except: pass
    for row in mask_T_list:
        try:
            xmin = row.index(boundary)
            xmax = len(row)-row[::-1].index(boundary)-1
            if y1>xmin: y1=xmin
            if y2<xmax: y2=xmax        
        except: pass
    return x1, y1, x2, y2

def plotGMM(inputs, n_components=5, SHOW_PLOT=False):
    """
    inputs - preds_dist[c_idx]
    """
    X = inputs.flatten().reshape(-1,1).cpu()
    # fit GMM
    gmm = GaussianMixture(n_components=n_components, init_params='kmeans', random_state=0, tol=1e-9)#, max_iter=n)
    with ignore_warnings(category=ConvergenceWarning):
        gmm.fit(X)

    y=[]
    # ------------- get x-range 중요 ---------------
    i_max = gmm.means_.argmax()
    i_min = gmm.means_.argmin()
    mu_max = gmm.means_[i_max][0]
    std_max = math.sqrt(gmm.covariances_[i_max][0])
    mu_min = gmm.means_[i_min][0]
    std_min = math.sqrt(gmm.covariances_[i_min][0])
    x = np.linspace(mu_min - 3*std_min, mu_max + 3*std_max, 200)
    # ------------- get x-range 중요 ---------------
    for i in range(n_components):
        mu = gmm.means_[i][0]
        var = gmm.covariances_[i][0]
        sigma = math.sqrt(var)
#         x = np.linspace(0.3,0.9,200)
        y.append(stats.norm.pdf(x, mu, sigma)*gmm.weights_[i])
        plt.plot(x, y[i])
#         x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
#         y.append(x)
#         plt.plot(x, stats.norm.pdf(x, mu, sigma)*gmm.weights_[i])
    # find decision boundary
    k = gmm.predict(x.reshape(-1,1))
    idx = np.where(np.abs(k[1:]-k[:-1])!=0) # 두 responsibility가 교체되는 x값의 인덱스
#     print(np.abs(k[1:]-k[:-1])!=0, idx)
    boundaries = x[1:][idx]
#     # boundaries 가 여러개 나올 수 있는데 그 중 likelihood가 가장 높은 값의 인덱스 선택
#     b_idx = gmm.score_samples(boundaries.reshape(-1,1)).argmax()  
#     boundary = boundaries[b_idx]
    # boundaries 가 여러개 나올 수 있는데 그 중 score 0.05이상의 가장 큰 값 선택
    boundary = boundaries[gmm.score_samples(boundaries.reshape(-1,1))>np.log(0.1)][-1]
    plt.axvline(x=boundary, c='r') 
    
    if(SHOW_PLOT):
        print(boundaries, gmm.score_samples(boundaries.reshape(-1,1)))
        sns.distplot(inputs.flatten().cpu().numpy())
        plt.show()
    return round(boundary, 3), gmm

# _, gmm = plotGMM(preds_dist[targets_gt[0]], n_components=5, SHOW_PLOT=True)

# import matplotlib.font_manager as fm

# font_path = 'C:\\Windows\\Fonts\\NanumBarunGothic.ttf'
# font_name = fm.FontProperties(fname=font_path, size=50).get_name()
# print(font_name)
# plt.rc('font', family=font_name)

# # font setting
# font_title = {'family': 'NanumBarunGothic', 'size': 18, 'color': 'black'}
# font_label = {'family': 'NanumBarunGothic', 'size': 12, 'color': 'black'}

def textureDetect(featuremaps, filter_size=(3,3), stride=(1,1), upsample=False):
    """
    box maker
    batch N - featuremaps.shape[0]
    """
    # upsample featuremap
    boxes_t = []
    esp = 1e-4
    # x,y 순서 바뀌면 안됨
    y_range = np.arange(0,featuremaps.shape[2]-1+esp, stride[0])
    x_range = np.arange(0,featuremaps.shape[3]-1+esp, stride[0])
    for y in y_range:
        for x in x_range:
            xx = x-((filter_size[0]-1)/2)
            yy = y-((filter_size[1]-1)/2)
            ww = filter_size[0]
            hh = filter_size[1]
            boxes_t.append([xx,yy,xx+ww,yy+hh]) # x,y,x2,y2
    boxes_t = torch.tensor(boxes_t).data.cuda().float()

    print('<featuremaps-shape>', featuremaps.shape)
    crops = torchvision.ops.roi_align(featuremaps, [boxes_t]*featuremaps.shape[0], [1,1]).data
    print('<roi-align>', crops.squeeze().shape)
    
    # inference using cosine similarity via class mean vector
    preds_dist = my_cos(crops.squeeze(), model.embedded_means)
    print('<class, roi-shape>', preds_dist.shape, len(y_range), len(x_range), '-->  (224,224)')

    preds_dist = preds_dist.transpose(1,0).reshape(-1, len(y_range), len(x_range))
    # upsamopling to same size for batch process
    if(upsample):
        preds_dist = nn.UpsamplingBilinear2d([224,224])(preds_dist.unsqueeze(0)).squeeze()
    print('[n_box]:', len(boxes_t))
    return preds_dist

def textureDetectMultiChannel(featuremaps, stride=(1,1), upsample=False):
    """
    box maker
    batch N - featuremaps.shape[0]
    """
    boxes_t = []
    base_boxes = []
    esp = 1e-4
#     channels = [(1,1), (3,3), (5,5), (7,7)]
    channels = [(1,1),(2,1),(1,2), (3,3),(6,3),(3,6)]#, (5,5),(5,10),(10,5)]
    # ------------------- 속도 최적화 필요 ----------------------
    # x,y 순서 바뀌면 안됨
    #     for filter_size in channels:
    y_range = np.arange(0,featuremaps.shape[2]-1+esp, stride[0])
    x_range = np.arange(0,featuremaps.shape[3]-1+esp, stride[0])
    for y in y_range:
        for x in x_range:
            xx = x+1/2
            yy = y+1/2
            xx2 = x+1/2
            yy2 = y+1/2
            base_boxes.append([xx,yy,xx2,yy2]) # x,y,x2,y2
    base_boxes = torch.tensor(base_boxes).data.cuda().float()
    
    for filter_size in channels:
        b = base_boxes.clone()
        b[:,0] = b[:,0]-filter_size[0]/2
        b[:,1] = b[:,1]-filter_size[1]/2
        b[:,2] = b[:,2]+filter_size[0]/2
        b[:,3] = b[:,3]+filter_size[1]/2
        boxes_t.append(b)
    boxes_t = torch.cat(boxes_t)
    # ------------------- 속도 최적화 필요 ----------------------
    
#     print('<featuremaps-shape>', featuremaps.shape)
    crops = torchvision.ops.roi_align(featuremaps, [boxes_t]*featuremaps.shape[0], [1,1]).data
#     print('<roi-align>', crops.squeeze().shape)
    
    # inference using cosine similarity via class mean vector
    preds_dist = my_cos(crops.squeeze(), model.embedded_means) # ch*h*w, cls
#     print('<class, roi-shape>', preds_dist.shape, len(channels), len(y_range), len(x_range), '-->  (224,224)')
    preds_dist = preds_dist.transpose(1,0).reshape(-1, len(channels), len(y_range), len(x_range)) # cls, ch*h*w
    # upsampling to same size for batch process
    if(upsample):
        preds_dist = nn.UpsamplingBilinear2d([224,224])(preds_dist)
#     print('[n_box]:', len(boxes_t))
    return preds_dist # cls,ch,h,w


def plotHeatMap(preds):
    fig = plt.figure(figsize=(20,15))
    for j in range(4):
        for i in range(8):
            n_fig = i+j*8+1
            if(n_fig==32):
                break
            a=fig.add_subplot(4,8,n_fig)
            imgplot = plt.imshow(preds_dist.cpu()[n_fig-1], cmap='jet')
            imgplot.set_clim(0,1)
            a.set_title(str(n_fig-1)+'//'+voc_val_dataset.classes[n_fig-1].split(' ')[0])
    plt.colorbar(ticks=[0.1,0.3,0.5,0.7,0.9], orientation ='horizontal')
    plt.show()
    return

def drawTextureRect(orig_fmap=None, fmap_box=None, box=None, n=4, filter_size=(3,3), stride=(.25, .25), isFirst=True, c=(0,0,255), EROSION=False):
    """
    filter_size - 클수록 멀리봄 
    stride - 작을수록 분해능 높음
    EROSION - 노이즈 제거
    """
    global featuremaps, frame
    """box maker"""
    if(isFirst):
        preds_dist = textureDetect(featuremaps, filter_size=filter_size, stride=stride)
        #     plotHeatMap(preds_dist)
    else:
        preds_dist = textureDetect(orig_fmap, filter_size=filter_size, stride=stride)
        #     plotHeatMap(preds_dist)

    print('<>', featuremaps.shape, preds_dist.shape)
    """use gmm"""
    boundary, gmm = plotGMM(preds_dist[targets_gt[0]], n_components=n, SHOW_PLOT=False)
    print('[boundary]:', boundary)

    mask = preds_dist[targets_gt[0]]
    mask = torch.where(mask>boundary, torch.zeros_like(mask).fill_(255), torch.zeros_like(mask))
    mask = mask.cpu().numpy()
    
    if(EROSION):
        kernel = np.ones(filter_size)
        mask = cv2.erode(mask.cpu().numpy(), kernel, iterations=3)
    
#     plt.imshow(mask)
#     plt.show()
    #     print(mask.shape)
    box = getOrdi(mask, 255) # x1, y1, x2, y2
    x1, y1, x2, y2 = box
    if(isFirst):
        r_y = frame.shape[0]/mask.shape[0]
        r_x = frame.shape[1]/mask.shape[1]
        x1*=r_x; x2*=r_x; y1*=r_y; y2*=r_y
    else:
        r_y = fmap_box[3]/mask.shape[0]
        r_x = fmap_box[2]/mask.shape[1]
        x1=fmap_box[0]+x1*r_x; x2=fmap_box[0]+x2*r_x; y1=fmap_box[1]+y1*r_y; y2=fmap_box[1]+y2*r_y
    try:
        print(boxes_gt, (int(x1), int(y1), int(x2-x1), int(y2-y1)))
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), c, 1, cv2.LINE_AA)
        cv2.putText(frame, 'n:'+str(n), (int(x1)-3, int(y1)-12), cv2.FONT_HERSHEY_SIMPLEX, .5, (50,50,50), 1)
    except: pass
    return (x1,y1,x2-x1,y2-y1), preds_dist # xywh


In [10]:
def cvt_yolobox(bboxes_yolo, size):
    """
    size - original [w,h]
    np.array
    yolo(cx,cy,w,h) to x,y,w,h
    """
    bboxes = bboxes_yolo.copy() # x,y,w,h

    bboxes[:,0] = (bboxes_yolo[:,0] - bboxes_yolo[:,2]/2)*size[1] # x
    bboxes[:,1] = (bboxes_yolo[:,1] - bboxes_yolo[:,3]/2)*size[0] # y
    bboxes[:,2] = bboxes_yolo[:,2]*size[1]
    bboxes[:,3] = bboxes_yolo[:,3]*size[0]
    bboxes = bboxes.astype(np.int)
    return bboxes

class Sobel(torch.nn.Module):
    def __init__(self):
        super(Sobel, self).__init__()
        sobel_x=torch.tensor([[-1, 0, 1],[-2,0,2],[-1,0,1]]).float().unsqueeze(0).unsqueeze(0)
        self.conv1=nn.Conv2d(1,1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv1.weight=nn.Parameter(sobel_x)
        
        sobel_y=torch.tensor([[1, 2, 1],[0,0,0],[-1,-2,-1]]).float().unsqueeze(0).unsqueeze(0)
        self.conv2=nn.Conv2d(1,1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2.weight=nn.Parameter(sobel_y)
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        
    def forward(self, x):
        """
        x - b,3ch(R,G,B),h,w
        return - b,1,h,w
        """
        # RGB to gray
        r, g, b = torch.chunk(x, chunks=3, dim=-3)
        gray = 0.299 * r + 0.587 * g + 0.110 * b

        G_x=self.conv1(gray).data
        G_y=self.conv2(gray).data

        G=torch.sqrt(torch.pow(G_x,2)+ torch.pow(G_y,2))
        return G
    
class Flatten(torch.nn.Module):
    """
    torch.nn.Sequential에서 사용가능한 flatten 모듈
    """
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)


class Bottleneck(nn.Module):
    def __init__(self, inchannel, outchannel, dilation=1, downsample=False):
        super(Bottleneck, self).__init__()
        self.downsample = downsample
        
        self.conv1 = nn.Conv2d(inchannel, inchannel, kernel_size=(1,1), stride=(1, 1))
        self.bn1 = nn.BatchNorm2d(inchannel, eps=1e-05, momentum=0.1, affine=True)
        self.conv2 = nn.Conv2d(inchannel, inchannel, kernel_size=(3,3), stride=(1, 1), padding=dilation, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(inchannel, eps=1e-05, momentum=0.1, affine=True)
        self.conv3 = nn.Conv2d(inchannel, outchannel, kernel_size=(1,1), stride=(1, 1))
        self.bn3 = nn.BatchNorm2d(outchannel, eps=1e-05, momentum=0.1, affine=True)
        self.pool = nn.MaxPool2d(2,2)
        self.relu = nn.ReLU()
        self.identity_conv = nn.Conv2d(1, outchannel, kernel_size=(1,1), stride=(1, 1))
        
        self.net = nn.Sequential(
            self.conv1,
            self.bn1,
            nn.ReLU(),
            self.conv2,
            self.bn2,
            nn.ReLU(),
            self.conv3,
            self.bn3,
        )
        
    def forward(self, x):
        identity = self.identity_conv(x[:,3:4]) # b*cls,edge_ch,224,224
        out = self.net(x)
        if(self.downsample):
            identity = self.pool(identity)
            out = self.pool(out)
        out += identity
        out = self.relu(out)
        return out
        
        
class BoxMaker(nn.Module):
    def __init__(self):
        super(BoxMaker, self).__init__()
        avg_pool = torch.nn.AdaptiveAvgPool2d(output_size=(1, 1))
        flatten = Flatten()
        self.roi_upsample = nn.UpsamplingBilinear2d([224,224])
        
        # backbone layer
        self.backbone = nn.Sequential(
            # 1x1 conv
            nn.Conv2d(6, 32, kernel_size=(3,3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, kernel_size=(3,3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, kernel_size=(3,3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(128, 256, kernel_size=(3,3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(), # n x 256 x 24 x 24
            avg_pool, # n x 256 x 1 x 1
            flatten, # n x 256
        )
        
        self.backbone2 = nn.Sequential(
            # 1x1 conv
            nn.Conv2d(6, 32, kernel_size=(3,3), stride=(1, 1), padding=(2,2), dilation=2),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, kernel_size=(3,3), stride=(1, 1), padding=(2,2), dilation=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, kernel_size=(3,3), stride=(1, 1), padding=(2,2), dilation=2),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(128, 256, kernel_size=(3,3), stride=(1, 1), padding=(2,2), dilation=2),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU(), # n x 256 x 24 x 24
            avg_pool, # n x 256 x 1 x 1
            flatten, # n x 256
        )
        
#         self.multihead_attn = nn.MultiheadAttention(embed_dim=512, num_heads=8)
                
        # fc layer
        self.fc = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 5), # score_logit + cx,cy,w,h
        )
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        
    def forward(self, inputs):
        """
        inputs - b,cls,ch,224,224
        """
        n_batch = inputs.shape[0] # b
        n_channel = inputs.shape[2] # ch
        # CNN backbone
        images = inputs.transpose(0,1).reshape(-1,n_channel,224,224) # b*cls,ch,224,224
        
        hh = self.backbone(images) # b*cls,256 
        hh2 = self.backbone2(images) # b*cls,256 
        hh3 = torch.cat([hh,hh2], dim=1) #b*cls,512
        hh3 = hh3.reshape(-1, n_batch, 512) # cls,b,512
        
#         attn_output, attn_output_weights = self.multihead_attn(hh3, hh3, hh3) # q,k,v - L,N,E - cls,b,512        
#         attn_sum = hh3+attn_output # cls,b,512
        
#         res = attn_sum.transpose(0,1) # b,cls,512
        res = hh3.transpose(0,1) # b,cls,512

        res2 = res.reshape(-1,512) # b*cls,512
        
        pp = self.fc(res2) # b*cls,5(1+4)
        pp = pp.reshape(n_batch,-1,5) # b,cls,5(1+4)
        logit = pp[:,:,0] # b,cls
        coords = pp[:,:,1:] # b,cls,4
        
        return logit, coords

sobel = Sobel()
sobel.to(sobel.device)
boxMaker = BoxMaker()
boxMaker.to(boxMaker.device)
# boxMaker.load_state_dict(torch.load('torch_models/heatmap-4chan-1130-2.pth', map_location='cuda:0'))
# # boxMaker.load_state_dict(torch.load('torch_models/heatmap-edge2-1129.pth', map_location='cuda:0'))
# boxMaker.eval()

BoxMaker(
  (roi_upsample): UpsamplingBilinear2d(size=[224, 224], mode=bilinear)
  (backbone): Sequential(
    (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2

---
#  
#  
# mAP

In [11]:
def concatSobel(preds_dist, data):
    """
    기존 히트맵 채널들과 sobel edge채널 합치기
    preds_dist - cls,ch,h,w
    data - b,3(RGB),h,w
    return - cls,ch+1,h,w
    """
    _data = nn.UpsamplingNearest2d([224,224])(data)
    data_sobel = sobel(_data.cuda())
    k = torch.stack([data_sobel[0]]*preds_dist.shape[0])
    return torch.cat([preds_dist, k], dim=1)

# concatSobel(preds_dist, data).shape

def calc_tp_state(boxes_ans, boxes_model, confidiences_model, filename=None, threshold_IOU=0.5):
    """
    average precision 계산용 데이터 프레임 리턴
    @params boxes - x,y,w,h
    state_array = [[filename, conf, state, iou], [filename, conf, state, iou], ...]
    threshold_IOU = 0.5 # 이걸 넘어야 TP, 못넘으면 FP
    """
    state_array = []
    for model_box_idx, model_box in enumerate(boxes_model):
        state = [filename, confidiences_model[model_box_idx], 0, 0] # default가 FP
        for ans_box_idx, ans_box in enumerate(boxes_ans):
            _IOU = float(calcIOU(model_box, ans_box))
            if _IOU>threshold_IOU: # IOU넘는게 하나라도 있으면 TP인 경우
                state = [filename, confidiences_model[model_box_idx], 1, _IOU] # default가 FP
                break
            if _IOU>state[3]: state[3]=_IOU # FP경우에도 최고 IOU기록
        state_array.append(state)
    _df = pd.DataFrame(state_array, columns=['image', 'confidience', 'TP', 'IOU'])
    return _df

from collections import Counter
# https://github.com/rafaelpadilla/Object-Detection-Metrics

def calc_AP(dataset_classes, label, target_all, dfs):
    target = dataset_classes.index(label)
    n_gt = Counter(target_all)

    _df = dfs[label].sort_values(by='confidience', ascending=False) # sort by conf

    _df['FP'] = [0 if tp==1 else 1 for tp in _df['TP']]
    _df['cum_TP'] = _df['TP'].cumsum()
    _df['cum_FP'] = _df['FP'].cumsum()
    _df['precision'] = [tp/(tp+fp) for tp,fp in zip(_df['cum_TP'],_df['cum_FP'])]
    _df['recall'] = _df['cum_TP']/n_gt[target]
    _df['precision_interpolated'] = [_df['precision'][i:].max() for i in range(len(_df['recall']))] 
    _df.reset_index(inplace=True)

    precisions = [_df['precision_interpolated'][_df['recall'].between(0.1*(i-1),0.1*i,inclusive=True)].min() for i in range(11)]
    precisions[0] = 1
    precisions = np.array([0 if p is np.nan else p for p in precisions])
    AP = precisions.mean()
#     print('[class]:', label, ', [AP]:', AP)
#     print('[Precisions]', precisions)

#     plt.plot(_df['recall'], _df['precision'])
#     plt.plot(_df['recall'], _df['precision_interpolated'], color='r')
#     plt.grid()
#     plt.xlim([0,1]), plt.title(label+' 11 point p-r curve')
#     plt.xlabel('recall'), plt.ylabel('precision')
#     plt.show()

    # display(_df)
    return AP, _df

In [31]:
import time
from datetime import datetime
#  ref - floor, frontview, office // val - floor2, frontview2, wall
# validation_21, reference_21_train // reference_21_val // all_10
# 21 10 바꾸기 주의, MyDataset도 바꿔야함 -10 해야함
root_dir = './references/all_10'
class_path = './references/obj_10.names'
# 21 10 바꾸기 주의, MyDataset도 바꿔야함 -10 해야함
# root_dir = './references/validation_21'
# class_path = './references/obj_21.names'

# ---------------------------- val set 로드 ---------------------------------
voc_val_dataset = MyDataset(root_dir, class_path, 
        transform=transforms.Compose([
#         myRotation,
#         transforms.RandomRotation(90, expand=True),
        transforms.ToTensor()
    ]))

voc_transform = transforms.Compose([
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

voc_classes = voc_val_dataset.classes
# assert val_dataset.classes==sample_dir_list, "샘플링 클래스와 데이터로더 클래스가 다름"
# ---------------------------- val set 정확도 측정 ---------------------------------

cols = ['timestamp', 'model', 'ref', 'val', 'similarity', 'shot', 'train-data', 'memo', 'acc', 'mAP', 'IOU', 
        *['AP'+str(c) for c in range(10)]]
savePath = 'mAP-result-log-1218.csv'
try:
    res_df = pd.read_csv(savePath)
except:
    res_df = pd.DataFrame(columns=cols)

    
fps=0
SHOW_IMAGE = False # 이미지 볼지 말지
# USE_KNN = False

boxMakerPath = 'heatmap-3chan-1219.pth'
# boxMakerPath = 'heatmap-6chan-1216.pth'
# boxMakerPath = 'heatmap-6chan-1218-21.pth'
# boxMakerPath = 'heatmap-7chan-1217.pth' # edge
# boxMakerPath = 'heatmap-9chan-1219.pth'

boxMaker.load_state_dict(torch.load('torch_models/'+boxMakerPath, map_location='cuda:0'))
boxMaker.eval()

memo = 'decay 0.1 per 2 chance'
trainData = 'voc 11'
similarity = 'cosine'

"""
전체 피처 데이터 재생성
"""
for ii in range(15):
    for shot in tqdm([10, 5, 3 ,1]):
        print('--------------- iter:', ii, 'shot:', shot, '---------------')
        startTime = time.time()

#         # 1,3 shot은 고른걸로
#         if shot in [1]: 
#             if ii==0: referenceImgDirPath = './references/images_trimed_10-%dshot'%shot # 10 classe
#             else: continue
#         else: referenceImgDirPath = './references/images_trimed_10-10shot' # 10 classes
# #         referenceImgDirPath = './references/images_trimed_21' # 21 classes
        if shot in [1,3,5]: referenceImgDirPath = './references/images_trimed_10-10shot' # 21 classes
        else: referenceImgDirPath = './references/images_trimed_10' # 21 classes      

        featuresPath = 'cosmetic_features_cos.csv'
        meanPath = 'cosmetic_means_cos.csv'
        makeAllReferenceCSV(referenceImgDirPath, featuresPath, meanPath, n_sample=shot)


        # -------------- n개 클래스만 validation 샘플링 세팅 및 폴더정리 ----------------
        reference_dataset = torchvision.datasets.ImageFolder(
            root=referenceImgDirPath,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
        sample_dir_list = reference_dataset.classes

        # -------------- 레퍼런스 데이터에서 n개 클래스 임베딩 & 레이블 & means 추출 -----------------
        featuresPath = 'cosmetic_features_cos.csv'
        meanPath = 'cosmetic_means_cos.csv'

        df_ref_featere_sampled, reference_means_sampled = get_sample_reference(featuresPath, meanPath, showData=False)

        # ---------------------------- 레퍼런스셋 로드 및 임베딩 세팅 ---------------------------------
        model.setReferenceDataset(sample_dir_list, df_ref_featere_sampled, reference_means_sampled)



        # ---------------detection-------------
        target_all_mAP = []
        ious = []
        dfs = {c:pd.DataFrame(columns=['image', 'confidience', 'TP', 'IOU']) for c in voc_classes} # average precision 저장
        correct = 0
        
        # print('[num_class]:', len(sample_dir_list), sample_dir_list)
        for i in tqdm(np.random.permutation(len(voc_val_dataset))):
        # for i in tqdm(range(len(voc_val_dataset))):
            data, targets_gt, boxes_gt, boxes_gt_yolo = voc_val_dataset[i] # 정답 데이터, 레이블, bbox
            target_all_mAP = np.concatenate([target_all_mAP,  targets_gt])

            frame = data.clone().detach().mul(255).squeeze().numpy().astype(np.uint8).transpose([1,2,0])
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # opencv image need to convert BGR -> RGB

            im_tensor = voc_transform(data.squeeze()).to(model.device).data.unsqueeze(0)
            featuremaps = model(im_tensor)

            preds_dist = textureDetectMultiChannel(featuremaps, stride=(0.25, 0.25), upsample=True)


#             logit, boxes_adj = boxMaker(concatSobel(preds_dist, data).unsqueeze(0).data)
            logit, boxes_adj = boxMaker(preds_dist.unsqueeze(0).data)
            pred_d, pred = torch.max(logit, 1)
        #     print('[Logit]', pred_d.float().sigmoid().item())
            boxes_adj = cvt_yolobox(boxes_adj[:,pred.item()].detach().cpu().numpy(), frame.shape[:2])
            iou = torchvision.ops.box_iou(box_cvt(torch.tensor(boxes_gt).cuda()), 
                                box_cvt(torch.tensor(boxes_adj).cuda())).item()
            ious.append(iou)
        #     print('[IOU]', iou)
            isRight = pred.item()==targets_gt[0]
            if isRight: correct+=1
#             res = str(isRight)+' [GT]:'+str(targets_gt[0])+' [Pred]:'+str(pred.item())+' - '+str(pred_d.float().sigmoid().item())[:5]
        #     print(res)  


            preds_np = pred.cpu().numpy()     
            for t in set(preds_np): # iterate all class // 현재는 하나만
                keep_gt = targets_gt==t
                keep_p = preds_np==t
                _df = calc_tp_state(boxes_gt[keep_gt], boxes_adj[keep_p], pred_d.detach().float().sigmoid().cpu().numpy()[keep_p], filename='-', threshold_IOU=0.5) # class 하나만 가정, 
                _class = voc_classes[t]
                dfs[_class] = dfs[_class].append(_df, ignore_index=True) # update df             
        
        # ------------- acc --------------
        acc = correct/len(voc_val_dataset)

        iou_mean = sum(ious)/len(ious)
        # ----------------------ap------------------------    
        AP_list = []
        df_res_list = {}
        for cls in voc_classes:
            AP, df_res = calc_AP(voc_classes, cls, target_all_mAP, dfs)
            AP_list.append(AP)
            df_res_list[cls] = df_res
        AP_list = np.array(AP_list)

        mAP = AP_list.mean()

        endTime = time.time()
        print('[Duration]:', int(endTime-startTime)//60, 'min', int(endTime-startTime)%60, 'sec')
        print('[SHOT]:',shot, '[Acc]:',acc, '[mAP]:', mAP, '[IOU-mean]:',iou_mean)        
        for c,ap in zip(voc_classes, AP_list):
            print(c, '-', str(ap)[:6])

        # save result
        result = pd.Series([str(datetime.now())[:19], boxMakerPath, referenceImgDirPath, root_dir, similarity,
                            shot, trainData, memo, acc, mAP, iou_mean, *AP_list], index=cols)
        res_df = res_df.append(pd.Series(result, index=cols), ignore_index=True)
        res_df.to_csv(savePath,index=False)
        res_df.to_csv(savePath+'.bak',index=False)    

A Jupyter Widget

--------------- iter: 0 shot: 10 ---------------


A Jupyter Widget

A Jupyter Widget

Exception in thread Thread-66:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tqdm\_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "C:\ProgramData\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



...

A Jupyter Widget

[Duration]: 1 min 28 sec
[SHOT]: 3 [Acc]: 0.5753286147623863 [mAP]: 0.34240419927412097 [IOU-mean]: 0.4743227746591435
AHC-AGELESS REAL EYE CREAM FOR FACE - 0.0925
AHC-Aura Secret Toneup Cream - 0.3875
AHC-ONLY FOR MAN LOTION - 0.5382
APIEU-데카소사이드 시카 겔 데이크림 - 0.4560
APIEU-스타트업 포어 프라이머 - 0.1003
BANILACO-프라임 프라이머 클래식 - 0.2644
BELIF-The true cream AQUA BOMB - 0.3107
CLIO-스파클링 라인 프리즘 에어 아이섀도우 - 0.3864
FERRAGAMO-INCANTO CHARMS - 0.5576
FERRARI-SCUDERIA BLACK EAU DE TOILETTE SPRAY - 0.3300
--------------- iter: 14 shot: 1 ---------------


A Jupyter Widget

A Jupyter Widget

[Duration]: 1 min 28 sec
[SHOT]: 1 [Acc]: 0.4914054600606673 [mAP]: 0.23850940326412612 [IOU-mean]: 0.4441865388062925
AHC-AGELESS REAL EYE CREAM FOR FACE - 0.0971
AHC-Aura Secret Toneup Cream - 0.2474
AHC-ONLY FOR MAN LOTION - 0.2355
APIEU-데카소사이드 시카 겔 데이크림 - 0.1212
APIEU-스타트업 포어 프라이머 - 0.1375
BANILACO-프라임 프라이머 클래식 - 0.3150
BELIF-The true cream AQUA BOMB - 0.1356
CLIO-스파클링 라인 프리즘 에어 아이섀도우 - 0.2610
FERRAGAMO-INCANTO CHARMS - 0.6097
FERRARI-SCUDERIA BLACK EAU DE TOILETTE SPRAY - 0.2246



In [14]:
import time
from datetime import datetime
#  ref - floor, frontview, office // val - floor2, frontview2, wall
# validation_21, reference_21_train // reference_21_val // all_10
# 21 10 바꾸기 주의, MyDataset도 바꿔야함 -10 해야함
root_dir = './references/all_2'
class_path = './references/obj_2.names'
# 21 10 바꾸기 주의, MyDataset도 바꿔야함 -10 해야함
# root_dir = './references/validation_21'
# class_path = './references/obj_21.names'

# ---------------------------- val set 로드 ---------------------------------
voc_val_dataset = MyDataset(root_dir, class_path, 
        transform=transforms.Compose([
#         myRotation,
#         transforms.RandomRotation(90, expand=True),
        transforms.ToTensor()
    ]))

voc_transform = transforms.Compose([
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

voc_classes = voc_val_dataset.classes
# assert val_dataset.classes==sample_dir_list, "샘플링 클래스와 데이터로더 클래스가 다름"
# ---------------------------- val set 정확도 측정 ---------------------------------

cols = ['timestamp', 'model', 'ref', 'val', 'similarity', 'shot', 'train-data', 'memo', 'acc', 'mAP', 'IOU',
                *['AP'+str(c) for c in range(2)]]
savePath = 'mAP-result-log-1218-attn2.csv'
try:
    res_df = pd.read_csv(savePath)
except:
    res_df = pd.DataFrame(columns=cols)

    
fps=0
SHOW_IMAGE = False # 이미지 볼지 말지
# USE_KNN = False

# boxMakerPath = 'heatmap-3chan-1219.pth'
boxMakerPath = 'heatmap-6chan-1223-no-attn.pth'
# boxMakerPath = 'heatmap-6chan-1218-21.pth'
# boxMakerPath = 'heatmap-7chan-1217.pth' # edge
# boxMakerPath = 'heatmap-9chan-1219.pth'

boxMaker.load_state_dict(torch.load('torch_models/'+boxMakerPath, map_location='cuda:0'))
boxMaker.eval()

memo = 'decay 0.1 per 2 chance'
trainData = 'voc 11'
similarity = 'cosine'
mmm = []
"""
전체 피처 데이터 재생성
"""
for ii in range(30):
    for shot in tqdm([1,3,5,10]):
        print('--------------- iter:', ii, 'shot:', shot, '---------------')
        startTime = time.time()

#         # 1,3 shot은 고른걸로
#         if shot in [1]: 
#             if ii==0: referenceImgDirPath = './references/images_trimed_10-%dshot'%shot # 10 classe
#             else: continue
#         else: referenceImgDirPath = './references/images_trimed_10-10shot' # 10 classes
# #         referenceImgDirPath = './references/images_trimed_21' # 21 classes
        if shot in [1,3,5]: referenceImgDirPath = './references/images_trimed_2-10shot' # 21 classes
        else: referenceImgDirPath = './references/images_trimed_2' # 21 classes      

        featuresPath = 'cosmetic_features_cos.csv'
        meanPath = 'cosmetic_means_cos.csv'
        makeAllReferenceCSV(referenceImgDirPath, featuresPath, meanPath, n_sample=shot)


        # -------------- n개 클래스만 validation 샘플링 세팅 및 폴더정리 ----------------
        reference_dataset = torchvision.datasets.ImageFolder(
            root=referenceImgDirPath,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
        sample_dir_list = reference_dataset.classes

        # -------------- 레퍼런스 데이터에서 n개 클래스 임베딩 & 레이블 & means 추출 -----------------
        featuresPath = 'cosmetic_features_cos.csv'
        meanPath = 'cosmetic_means_cos.csv'

        df_ref_featere_sampled, reference_means_sampled = get_sample_reference(featuresPath, meanPath, showData=False)

        # ---------------------------- 레퍼런스셋 로드 및 임베딩 세팅 ---------------------------------
        model.setReferenceDataset(sample_dir_list, df_ref_featere_sampled, reference_means_sampled)



        # ---------------detection-------------
        target_all_mAP = []
        ious = []
        dfs = {c:pd.DataFrame(columns=['image', 'confidience', 'TP', 'IOU']) for c in voc_classes} # average precision 저장
        correct = 0
        
        # print('[num_class]:', len(sample_dir_list), sample_dir_list)
        for i in tqdm(np.random.permutation(len(voc_val_dataset))):
        # for i in tqdm(range(len(voc_val_dataset))):
            data, targets_gt, boxes_gt, boxes_gt_yolo = voc_val_dataset[i] # 정답 데이터, 레이블, bbox
            target_all_mAP = np.concatenate([target_all_mAP,  targets_gt])

            frame = data.clone().detach().mul(255).squeeze().numpy().astype(np.uint8).transpose([1,2,0])
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # opencv image need to convert BGR -> RGB

            im_tensor = voc_transform(data.squeeze()).to(model.device).data.unsqueeze(0)
            featuremaps = model(im_tensor)

            preds_dist = textureDetectMultiChannel(featuremaps, stride=(0.25, 0.25), upsample=True)


#             logit, boxes_adj = boxMaker(concatSobel(preds_dist, data).unsqueeze(0).data)
            logit, boxes_adj = boxMaker(preds_dist.unsqueeze(0).data)
            pred_d, pred = torch.max(logit, 1)
        #     print('[Logit]', pred_d.float().sigmoid().item())
            boxes_adj = cvt_yolobox(boxes_adj[:,pred.item()].detach().cpu().numpy(), frame.shape[:2])
            iou = torchvision.ops.box_iou(box_cvt(torch.tensor(boxes_gt).cuda()), 
                                box_cvt(torch.tensor(boxes_adj).cuda())).item()
            ious.append(iou)
        #     print('[IOU]', iou)
            isRight = pred.item()==targets_gt[0]
            if isRight: correct+=1
#             res = str(isRight)+' [GT]:'+str(targets_gt[0])+' [Pred]:'+str(pred.item())+' - '+str(pred_d.float().sigmoid().item())[:5]
        #     print(res)  


            preds_np = pred.cpu().numpy()     
            for t in set(preds_np): # iterate all class // 현재는 하나만
                keep_gt = targets_gt==t
                keep_p = preds_np==t
                _df = calc_tp_state(boxes_gt[keep_gt], boxes_adj[keep_p], pred_d.detach().float().sigmoid().cpu().numpy()[keep_p], filename='-', threshold_IOU=0.5) # class 하나만 가정, 
                _class = voc_classes[t]
                dfs[_class] = dfs[_class].append(_df, ignore_index=True) # update df             
        
        # ------------- acc --------------
        acc = correct/len(voc_val_dataset)

        iou_mean = sum(ious)/len(ious)
        # ----------------------ap------------------------    
        AP_list = []
        df_res_list = {}
        for cls in voc_classes:
            AP, df_res = calc_AP(voc_classes, cls, target_all_mAP, dfs)
            AP_list.append(AP)
            df_res_list[cls] = df_res
        AP_list = np.array(AP_list)

        mAP = AP_list.mean()

        endTime = time.time()
        print('[Duration]:', int(endTime-startTime)//60, 'min', int(endTime-startTime)%60, 'sec')
        print('[SHOT]:',shot, '[Acc]:',acc, '[mAP]:', mAP, '[IOU-mean]:',iou_mean)        
        for c,ap in zip(voc_classes, AP_list):
            print(c, '-', str(ap)[:6])

        mmm.append(mAP)
#         # save result
#         result = pd.Series([str(datetime.now())[:19], boxMakerPath, referenceImgDirPath, root_dir, similarity,
#                             shot, trainData, memo, acc, mAP, iou_mean, *AP_list], index=cols)
#         res_df = res_df.append(pd.Series(result, index=cols), ignore_index=True)
#         res_df.to_csv(savePath,index=False)
#         res_df.to_csv(savePath+'.bak',index=False)    

A Jupyter Widget

--------------- iter: 0 shot: 1 ---------------


A Jupyter Widget

A Jupyter Widget

[Duration]: 0 min 45 sec
[SHOT]: 1 [Acc]: 0.9241706161137441 [mAP]: 0.5088249780898354 [IOU-mean]: 0.5107237977368572
AHC-ONLY FOR MAN LOTION - 0.3883
APIEU-데카소사이드 시카 겔 데이크림 - 0.6293
--------------- iter: 0 shot: 3 ---------------


...

A Jupyter Widget

[Duration]: 0 min 45 sec
[SHOT]: 5 [Acc]: 0.985781990521327 [mAP]: 0.6988482248659652 [IOU-mean]: 0.5729257039972956
AHC-ONLY FOR MAN LOTION - 0.7612
APIEU-데카소사이드 시카 겔 데이크림 - 0.6364
--------------- iter: 0 shot: 10 ---------------


A Jupyter Widget

A Jupyter Widget

KeyboardInterrupt: 

In [13]:
res_df

,timestamp,model,ref,val,similarity,shot,train-data,memo,acc,mAP,IOU,AP0,AP1
0,2019-12-22 17:42:36,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,1,voc 11,decay 0.1 per 2 chance,0.895735,0.468858,0.494971,0.397946,0.539771
1,2019-12-22 17:43:22,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,3,voc 11,decay 0.1 per 2 chance,0.981043,0.489954,0.536775,0.471694,0.508214
2,2019-12-22 17:44:08,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,5,voc 11,decay 0.1 per 2 chance,0.990521,0.553405,0.538578,0.609672,0.497139
3,2019-12-22 17:44:54,heatmap-6chan-1216.pth,./references/images_trimed_2,./references/all_2,cosine,10,voc 11,decay 0.1 per 2 chance,0.981043,0.508362,0.519684,0.527978,0.488746
4,2019-12-22 17:45:39,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,1,voc 11,decay 0.1 per 2 chance,0.952607,0.462381,0.508459,0.482064,0.442698
5,2019-12-22 17:46:25,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,3,voc 11,decay 0.1 per 2 chance,0.985782,0.548734,0.534870,0.629901,0.467566
6,2019-12-22 17:47:10,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,5,voc 11,decay 0.1 per 2 chance,0.990521,0.561403,0.543246,0.610180,0.512626
7,2019-12-22 17:47:56,heatmap-6chan-1216.pth,./references/images_trimed_2,./references/all_2,cosine,10,voc 11,decay 0.1 per 2 chance,0.990521,0.450853,0.529316,0.447112,0.454593
8,2019-12-22 17:48:42,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,1,voc 11,decay 0.1 per 2 chance,0.962085,0.508596,0.530790,0.646610,0.370582
9,2019-12-22 17:49:27,heatmap-6chan-1216.pth,./references/images_trimed_2-10shot,./references/all_2,cosine,3,voc 11,decay 0.1 per 2 chance,0.981043,0.579386,0.553185,0.686387,0.472384
